In [17]:
import argparse
import yaml
from easydict import EasyDict
from diffmot import DiffMOT
import torch
with open('configs/mot.yaml') as f:
    config = yaml.safe_load(f)

class Args:
    def __init__(self):
        self.config = 'configs/mot.yaml'
        self.dataset = 'mot'
        self.skip_connection = False
        self.network = 'ReUNet'
        self.filters = [16, 32, 64, 128, 256]
        self.num_layers = 1
        self.early_stopping = 'iou'
        self.patience = 5
        self.delta = 0
        self.batch_size = 4

    def items(self):
        return vars(self).items()
    
config = EasyDict(config)
args = Args()

if args.network is not None:
    config.network = args.network
if args.num_layers is not None:
    config.num_layers = args.num_layers
if args.filters is not None:
    config.filters = args.filters
if args.skip_connection is not None:
    config.skip_connection = args.skip_connection
if args.early_stopping is not None:
    config.early_stopping = args.early_stopping
config.patience = args.patience
config.delta = args.delta
config.batch_size = args.batch_size

config["exp_name"] = args.config.split("/")[-1].split(".")[0]
config["dataset"] = args.dataset
config = EasyDict(config)
agent = DiffMOT(config)
batch = next(iter(agent.train_dataloader))

2024-12-25 00:24:13 [INFO]: Config:
2024-12-25 00:24:13 [INFO]: {'eps': 0.001, 'eval_mode': False, 'patience': 5, 'delta': 0, 'lr': 0.00025, 'num_layers': 1, 'filters': [16, 32, 64, 128, 256], 'weight': False, 'weight_1': 0.25, 'weight_2': 0.25, 'weight_3': 0.5, 'loss': 'l2', 'data_dir': 'dataset/trackers_gt_t/train', 'diffnet': 'HMINet', 'interval': 5, 'augment': True, 'encoder_dim': 256, 'tf_layer': 3, 'epochs': 800, 'batch_size': 4, 'seed': 123, 'eval_every': 20, 'gpus': [0, 1, 2, 3], 'eval_at': 800, 'det_dir': '/mnt/8T/home/estar/data/DanceTrack/detections/val', 'info_dir': '/mnt/8T/home/estar/data/DanceTrack/val', 'reid_dir': '/home/estar/lwy/DiffMOT/cache/embeddings/', 'save_dir': '/mnt/8T/home/estar/data/DanceTrack/results/val/yolox_m_lt_ddm_1000eps_deeper_800_1rev', 'eval_expname': 'mot_ddm_1000_deeper', 'high_thres': 0.6, 'low_thres': 0.1, 'w_assoc_emb': 2.2, 'aw_param': 1.7, 'preprocess_workers': 16, 'device': 'cuda', 'eval_device': 'None', 'network': 'ReUNet', 'skip_connecti

> Directory built!
> Model built!
Network Version:  ReUNet
Network Params: 16172299
Dataset: dataset/trackers_gt_t/train
dataset summary
1230449
len:  1230449
Training Dataset size:  984359
Validation Dataset size:  246090
> Optimizer built!
> Everything built. Have fun :)


In [27]:
import torch
from utils import calculate_iou, calculate_ade, original_shape

num_batches = args.batch_size
total_loss = 0
total_iou = 0
total_ade = 0

for k in batch:
    batch[k] = batch[k].to(device=agent.device, non_blocking=True)

loss = agent.model(batch)
loss

tensor(0.0944, grad_fn=<MeanBackward0>)

In [25]:
loss = loss.mean()
total_loss += loss.item()

# MeanIoU and MeanADE
with torch.no_grad():
    predictions = agent.generate(conds = batch['condition'], sample = 1, bestof =  True, flexibility=0.0, ret_traj=False) # Batch_size, 4

dets = batch['condition'][:, 4, :4] # Batch_size, 4
targets = batch['cur_bbox'] # Batch_size, 4
width = batch['width'] # Batch_size
height = batch['height'] # Batch_size

predictions = predictions + dets # Batch_size, 4
original_preds = original_shape(predictions, width, height) # Batch_size, 4
original_gts = original_shape(targets, width, height) # Batch_size, 4

total_iou += calculate_iou(original_preds, original_gts)
total_ade += calculate_ade(original_preds, original_gts)

mean_loss = total_loss / num_batches
mean_iou = total_iou / num_batches
mean_ade = total_ade / num_batches

result = {
    'mean_loss': mean_loss,
    'mean_iou': mean_iou,
    'mean_ade': mean_ade
}

result

{'mean_loss': 0.007626724429428577,
 'mean_iou': 0.0,
 'mean_ade': 61.0211067199707}

# Test Model

In [1]:
import os
import glob

In [12]:
import os
import glob

def find_files_with_fewer_than_6_lines(directory):
    # Get all .txt files in the specified directory and its subdirectories
    txt_files = glob.glob(os.path.join(directory, '**', '*.txt'), recursive=True)
    
    # List to store files with fewer than 6 lines
    files_with_fewer_lines = []

    for txt_file in txt_files:
        txt_file = os.path.normpath(txt_file)
        with open(txt_file, 'r') as file:
            lines = file.readlines()
            if len(lines) < 6:
                files_with_fewer_lines.append(txt_file)

    return files_with_fewer_lines

# Specify the directory
directory = 'dataset/trackers_gt_t/train/'

# Find files with fewer than 6 lines
files_with_fewer_lines = find_files_with_fewer_than_6_lines(directory)

# Print the results
for file in files_with_fewer_lines:
    print(file)

dataset\trackers_gt_t\train\MOT17-02-DPM\img1\000080.txt
dataset\trackers_gt_t\train\MOT17-02-DPM\img1\000081.txt
dataset\trackers_gt_t\train\MOT17-02-DPM\img1\000082.txt
dataset\trackers_gt_t\train\MOT17-02-DPM\img1\000083.txt
dataset\trackers_gt_t\train\MOT17-05-DPM\img1\000010.txt
dataset\trackers_gt_t\train\MOT17-05-DPM\img1\000018.txt
dataset\trackers_gt_t\train\MOT17-05-DPM\img1\000021.txt
dataset\trackers_gt_t\train\MOT17-05-DPM\img1\000032.txt
dataset\trackers_gt_t\train\MOT17-05-DPM\img1\000037.txt
dataset\trackers_gt_t\train\MOT17-05-DPM\img1\000040.txt
dataset\trackers_gt_t\train\MOT17-05-DPM\img1\000064.txt
dataset\trackers_gt_t\train\MOT17-05-DPM\img1\000079.txt
dataset\trackers_gt_t\train\MOT17-05-DPM\img1\000148.txt
dataset\trackers_gt_t\train\MOT17-05-DPM\img1\000149.txt
dataset\trackers_gt_t\train\MOT17-05-DPM\img1\000151.txt
dataset\trackers_gt_t\train\MOT17-05-DPM\img1\000152.txt
dataset\trackers_gt_t\train\MOT17-05-DPM\img1\000154.txt
dataset\trackers_gt_t\train\MOT

In [4]:
txt_files = glob.glob(os.path.join(directory, '**', '*.txt'), recursive=True)
os.path.normpath(txt_files[0])

'dataset\\trackers_gt_t\\train\\MOT17-02-DPM\\img1\\000002.txt'

In [6]:
txt_file = os.path.normpath(txt_files[0])
txt_file

'dataset\\trackers_gt_t\\train\\MOT17-02-DPM\\img1\\000002.txt'

In [7]:
with open(os.path.normpath(txt_file), 'r') as file:
    lines = file.readlines()

In [10]:
len(lines)

56

In [2]:
train_dataloader = agent.train_dataloader
sample = next(iter(train_dataloader))

In [3]:
sample.keys()

dict_keys(['cur_gt', 'cur_bbox', 'condition', 'delta_bbox', 'width', 'height'])

In [4]:
sample['delta_bbox'].shape

torch.Size([2048, 4])

In [7]:
import torch

batch = {
    'condition' : torch.rand(16, 5, 8),
    'delta_bbox' : torch.rand(16, 4),
}

In [5]:
model = agent.model

In [8]:
loss = model(batch)

In [9]:
loss

tensor(1.3105, grad_fn=<MeanBackward0>)

In [18]:
model.diffusion.net.filters

[16, 64, 128, 256]

In [16]:
model.diffusion.net

New_ReUNet(
  (shared_ctx_mlp): MLP(
    (layer_norm): LayerNorm((259,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (dense_layer): Sequential(
      (0): Linear(in_features=259, out_features=518, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=518, out_features=259, bias=True)
      (4): ReLU()
      (5): Dropout(p=0.1, inplace=False)
      (6): Linear(in_features=259, out_features=256, bias=True)
    )
  )
  (up_blocks): ModuleList(
    (0): LSTMAoA(
      (out_dropout): Dropout(p=0.1, inplace=False)
      (norm): LayerNorm((260,), eps=1e-05, elementwise_affine=True)
      (att_lstm): LSTM(260, 256, num_layers=2)
      (multi_head): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
      )
      (aoa_layer): Sequential(
        (0): Linear(in_features=512, out_features=512, bias=True)
        (1): GLU(dim=-1)
      )
      (r